# Test of custom layers

In [1]:
import numpy as np
from functools import reduce
from operator import mul
import tensorflow as tf
tf.enable_eager_execution() 
m = tf.compat.v2.math
keras = tf.compat.v2.keras
K = keras.backend

In [38]:
# test setup
def test_layer(make_layer):
    VALUE_SHAPE = (3,)
    
    in_vals = [
        np.array([
            np.reshape(np.arange(0,reduce(mul, VALUE_SHAPE), dtype=np.dtype('float32')), VALUE_SHAPE)-10,
            np.reshape(np.arange(0,reduce(mul, VALUE_SHAPE), dtype=np.dtype('float32')), VALUE_SHAPE),
        ]),
        np.array([
            np.ones(VALUE_SHAPE, dtype=np.dtype('float32')),
            np.zeros(VALUE_SHAPE, dtype=np.dtype('float32'))
        ])
    ]
    
    out_val = make_layer()(in_vals)
    return out_val

In [39]:
# CCSA-keras implementation

def euclidean_distance(vects):
    eps = 1e-08
    x, y = vects
    return K.sqrt(K.maximum(K.sum(K.square(x - y), axis=1, keepdims=True), eps))

def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

def make_euc_dist():
    return keras.layers.Lambda(
        euclidean_distance, 
        output_shape=eucl_dist_output_shape, name='CSA'
    )

test_layer(make_euc_dist).numpy()

array([[17.378147],
       [ 2.236068]], dtype=float32)

In [45]:
# custom distance layer implementation
M = tf.compat.v2.math
class Distance(keras.layers.Layer):
    '''
    Frobenius Norm of distance between inputs.
    Assumes the input two be array-like with two elements
    Implementation only supports 1D data
    '''
    def __init__(self, name=None):
        super(Distance, self).__init__(name=name)

    def call(self, inputs):
        a = inputs[0]
        b = inputs[1]
        return M.square(M.sqrt(M.reduce_sum(M.square(a-b),axis=1)))

M.sqrt(test_layer(Distance).numpy()).numpy()

array([17.378147,  2.236068], dtype=float32)

In [23]:
ld = [
    {'a':1, 'b':10},
    {'a':2, 'b':20},
]

In [27]:
reduce(lambda dn, do: {k:dn[k]+do[k] for k in dn},ld, {k:0 for k in ld[0]})

{'a': 3, 'b': 30}

In [28]:
ll = [
    [1,10],
    [2,20]
]

In [30]:
reduce(
    lambda n, o: [n[i]+o[i] for i in range(len(ll))],
    ll, 
    [0 for _ in ll[0]]
)

[3, 30]

In [101]:
bs = 2 #batch size
es = 3 # embed size
xs = tf.constant([[1,2,3],[4,5,6]], dtype=tf.float32)
xt = tf.constant([[3,2,1],[4,4,4]], dtype=tf.float32)

In [109]:
xs_rpt = tf.broadcast_to(tf.expand_dims(xs, axis=0),shape=(bs,bs,es),name=None)
xt_rpt = tf.broadcast_to(tf.expand_dims(xt, axis=1),shape=(bs,bs,es),name=None)

In [137]:
dists = tf.reduce_sum(tf.square(xs_rpt-xt_rpt), axis=2)
dists

<tf.Tensor: id=420, shape=(2, 2), dtype=float32, numpy=
array([[ 8., 35.],
       [14.,  5.]], dtype=float32)>

In [150]:
ys = tf.constant([0,1])
yt = tf.constant([1,2])

In [151]:
ys_rpt = tf.broadcast_to(tf.expand_dims(ys, axis=1), shape=(bs, bs))
yt_rpt = tf.broadcast_to(tf.expand_dims(yt, axis=0), shape=(bs, bs))

In [152]:
y_same = tf.cast(tf.equal(ys_rpt,yt_rpt), tf.float32)
y_diff = tf.cast(tf.not_equal(ys_rpt,yt_rpt), tf.float32)
y_same, y_diff

(<tf.Tensor: id=468, shape=(2, 2), dtype=float32, numpy=
 array([[0., 0.],
        [1., 0.]], dtype=float32)>,
 <tf.Tensor: id=470, shape=(2, 2), dtype=float32, numpy=
 array([[1., 1.],
        [0., 1.]], dtype=float32)>)

In [153]:
intra_cls_dists = tf.multiply(sum_square, y_same)
inter_cls_dists = tf.multiply(sum_square, y_diff)
intra_cls_dists, inter_cls_dists

(<tf.Tensor: id=473, shape=(2, 2), dtype=float32, numpy=
 array([[ 0.,  0.],
        [14.,  0.]], dtype=float32)>,
 <tf.Tensor: id=474, shape=(2, 2), dtype=float32, numpy=
 array([[ 8., 35.],
        [ 0.,  5.]], dtype=float32)>)

In [154]:
max_dists = tf.reduce_max(dists, axis=1, keepdims=True)
max_dists = tf.broadcast_to(max_dists, shape=(bs, bs))
max_dists

<tf.Tensor: id=480, shape=(2, 2), dtype=float32, numpy=
array([[35., 35.],
       [14., 14.]], dtype=float32)>

In [155]:
y_same, max_dists, inter_cls_dists

(<tf.Tensor: id=468, shape=(2, 2), dtype=float32, numpy=
 array([[0., 0.],
        [1., 0.]], dtype=float32)>,
 <tf.Tensor: id=480, shape=(2, 2), dtype=float32, numpy=
 array([[35., 35.],
        [14., 14.]], dtype=float32)>,
 <tf.Tensor: id=474, shape=(2, 2), dtype=float32, numpy=
 array([[ 8., 35.],
        [ 0.,  5.]], dtype=float32)>)

In [158]:
revised_inter_cls_dists = tf.where(tf.cast(y_same, dtype=tf.bool), max_dists, inter_cls_dists)
revised_inter_cls_dists

<tf.Tensor: id=491, shape=(2, 2), dtype=float32, numpy=
array([[ 8., 35.],
       [14.,  5.]], dtype=float32)>

In [159]:
max_intra_cls_dist = tf.reduce_max(intra_cls_dists, axis=1)
min_inter_cls_dist = tf.reduce_min(revised_inter_cls_dists, axis=1)
max_intra_cls_dist, min_inter_cls_dist

(<tf.Tensor: id=494, shape=(2,), dtype=float32, numpy=array([ 0., 14.], dtype=float32)>,
 <tf.Tensor: id=496, shape=(2,), dtype=float32, numpy=array([8., 5.], dtype=float32)>)

In [165]:
margin=0
tf.nn.relu(max_intra_cls_dist - min_inter_cls_dist + margin)

<tf.Tensor: id=518, shape=(2,), dtype=float32, numpy=array([0., 9.], dtype=float32)>

In [166]:
def ho(a):
    def fn(b):
        return a+b
    return fn

ho(2)(3)

5

In [172]:
ys=tf.constant([1,2,1])
yt=tf.constant([1,2,2])

In [176]:
tf.cast(tf.equal(ys,yt), dtype=tf.float32)

<tf.Tensor: id=528, shape=(3,), dtype=float32, numpy=array([1., 1., 0.], dtype=float32)>

In [177]:
ys

<tf.Tensor: id=520, shape=(3,), dtype=int32, numpy=array([1, 2, 1], dtype=int32)>

In [183]:
ys.shape[0]

Dimension(3)

In [184]:
tf.compat.v2.TensorShape([2,ys.shape[0]])

TensorShape([Dimension(2), Dimension(3)])

In [208]:
y_true = tf.constant([[[1,0,0],[0,1,0]], [[1,0,0],[0,0,1]]])
y_true

<tf.Tensor: id=638, shape=(2, 2, 3), dtype=int32, numpy=
array([[[1, 0, 0],
        [0, 1, 0]],

       [[1, 0, 0],
        [0, 0, 1]]], dtype=int32)>

In [219]:
ys, yt = y_true
ys, yt

(<tf.Tensor: id=683, shape=(2, 3), dtype=int32, numpy=
 array([[1, 0, 0],
        [0, 1, 0]], dtype=int32)>,
 <tf.Tensor: id=687, shape=(2, 3), dtype=int32, numpy=
 array([[1, 0, 0],
        [0, 0, 1]], dtype=int32)>)

In [228]:
tf.cast(tf.reduce_all(tf.equal(ys,yt),axis=1, keepdims=True), dtype=tf.float32)

<tf.Tensor: id=733, shape=(2, 1), dtype=float32, numpy=
array([[1.],
       [0.]], dtype=float32)>

In [216]:
ys-yt

<tf.Tensor: id=673, shape=(2, 3), dtype=int32, numpy=
array([[ 0,  0,  0],
       [ 0,  1, -1]], dtype=int32)>

In [231]:
K.square(ys)

<tf.Tensor: id=738, shape=(2, 3), dtype=int32, numpy=
array([[1, 0, 0],
       [0, 1, 0]], dtype=int32)>

# Graph Embedding

In [96]:
xs = tf.constant([[1,2,3],[4,5,6]], dtype=tf.float32)
xt = tf.constant([[3,2,1],[4,4,4]], dtype=tf.float32)
ys = tf.constant([0,1])
yt = tf.constant([0,1])

In [114]:
y_true = tf.constant([[[True, False, False], [True, False, False]],
                      [[False, False, True], [False, False, True]]])

y_pred = tf.constant([[[1,2,3],[4,5,6]],
                      [[3,2,1],[4,4,4]]], dtype=tf.float32)

In [130]:
ys = tf.argmax(tf.cast(y_true[:,0], dtype=tf.int32), axis=1)
yt = tf.argmax(tf.cast(y_true[:,1], dtype=tf.int32), axis=1)
xs = y_pred[:,0]
xt = y_pred[:,1]
θϕ = tf.transpose(tf.concat([xs,xt], axis=0))
ys, yt, xs, xt, θϕ

(<tf.Tensor: id=845, shape=(2,), dtype=int64, numpy=array([0, 2])>,
 <tf.Tensor: id=852, shape=(2,), dtype=int64, numpy=array([0, 2])>,
 <tf.Tensor: id=856, shape=(2, 3), dtype=float32, numpy=
 array([[1., 2., 3.],
        [3., 2., 1.]], dtype=float32)>,
 <tf.Tensor: id=860, shape=(2, 3), dtype=float32, numpy=
 array([[4., 5., 6.],
        [4., 4., 4.]], dtype=float32)>,
 <tf.Tensor: id=864, shape=(3, 4), dtype=float32, numpy=
 array([[1., 3., 4., 4.],
        [2., 2., 5., 4.],
        [3., 1., 6., 4.]], dtype=float32)>)

In [131]:
batch_size = ys.shape[0].value
batch_size

2

In [139]:
# make_weights: relate_all
N = 2*batch_size 
y = tf.concat([ys, yt], axis = 0) # TODO: verify axis
yTe = tf.broadcast_to(tf.expand_dims(y, axis=1), shape=(N, N))
eTy = tf.broadcast_to(tf.expand_dims(y, axis=0), shape=(N, N))

W = tf.equal(yTe, eTy)
Wp = tf.not_equal(yTe, eTy)
N, y, yTe, eTy, W, Wp

(4,
 <tf.Tensor: id=940, shape=(4,), dtype=int64, numpy=array([0, 2, 0, 2])>,
 <tf.Tensor: id=944, shape=(4, 4), dtype=int64, numpy=
 array([[0, 0, 0, 0],
        [2, 2, 2, 2],
        [0, 0, 0, 0],
        [2, 2, 2, 2]])>,
 <tf.Tensor: id=948, shape=(4, 4), dtype=int64, numpy=
 array([[0, 2, 0, 2],
        [0, 2, 0, 2],
        [0, 2, 0, 2],
        [0, 2, 0, 2]])>,
 <tf.Tensor: id=949, shape=(4, 4), dtype=bool, numpy=
 array([[ True, False,  True, False],
        [False,  True, False,  True],
        [ True, False,  True, False],
        [False,  True, False,  True]])>,
 <tf.Tensor: id=950, shape=(4, 4), dtype=bool, numpy=
 array([[False,  True, False,  True],
        [ True, False,  True, False],
        [False,  True, False,  True],
        [ True, False,  True, False]])>)

In [140]:
# make weights: relate source-target
N = 2*batch_size 
i = tf.constant([[False,True],[True,False]], dtype=tf.bool)
for ax in range(2):
    i = keras.backend.repeat_elements(i, N//2, axis=ax)

zeros = tf.zeros([N,N],dtype=tf.bool)
W = tf.where(i, W, zeros)
Wp = tf.where(i, Wp, zeros)
W, Wp

(<tf.Tensor: id=969, shape=(4, 4), dtype=bool, numpy=
 array([[False, False,  True, False],
        [False, False, False,  True],
        [ True, False, False, False],
        [False,  True, False, False]])>,
 <tf.Tensor: id=970, shape=(4, 4), dtype=bool, numpy=
 array([[False, False, False,  True],
        [False, False,  True, False],
        [False,  True, False, False],
        [ True, False, False, False]])>)

In [154]:
# make weights: relate source-target pair
eq = tf.linalg.diag(tf.equal(ys, yt))
neq = tf.linalg.diag(tf.not_equal(ys, yt))
zeros = tf.zeros([batch_size, batch_size],dtype=tf.bool)
W = tf.concat([tf.concat([zeros, eq],axis=0),
               tf.concat([eq, zeros],axis=0)], axis=1)
Wp = tf.concat([tf.concat([zeros, neq],axis=0),
                tf.concat([neq, zeros],axis=0)], axis=1)
eq, neq, W, Wp

(<tf.Tensor: id=1080, shape=(2, 2), dtype=bool, numpy=
 array([[ True, False],
        [False,  True]])>,
 <tf.Tensor: id=1082, shape=(2, 2), dtype=bool, numpy=
 array([[False, False],
        [False, False]])>,
 <tf.Tensor: id=1093, shape=(4, 4), dtype=bool, numpy=
 array([[False, False,  True, False],
        [False, False, False,  True],
        [ True, False, False, False],
        [False,  True, False, False]])>,
 <tf.Tensor: id=1099, shape=(4, 4), dtype=bool, numpy=
 array([[False, False, False, False],
        [False, False, False, False],
        [False, False, False, False],
        [False, False, False, False]])>)

In [155]:
W, Wp = tf.cast(W, dtype=tf.float32), tf.cast(Wp, dtype=tf.float32)

In [156]:
D  = tf.linalg.diag(tf.reduce_sum(W,  axis=1)) 
Dp = tf.linalg.diag(tf.reduce_sum(Wp, axis=1))
D, Dp

(<tf.Tensor: id=1104, shape=(4, 4), dtype=float32, numpy=
 array([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]], dtype=float32)>,
 <tf.Tensor: id=1107, shape=(4, 4), dtype=float32, numpy=
 array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]], dtype=float32)>)

In [157]:
L  = tf.subtract(D, W)
Lp = tf.subtract(Dp, Wp)
L, Lp

(<tf.Tensor: id=1108, shape=(4, 4), dtype=float32, numpy=
 array([[ 1.,  0., -1.,  0.],
        [ 0.,  1.,  0., -1.],
        [-1.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  1.]], dtype=float32)>,
 <tf.Tensor: id=1109, shape=(4, 4), dtype=float32, numpy=
 array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]], dtype=float32)>)

In [145]:
θϕLϕθ  = tf.matmul(θϕ, tf.matmul(L,  θϕ, transpose_b=True))
θϕLpϕθ = tf.matmul(θϕ, tf.matmul(Lp, θϕ, transpose_b=True))
θϕLϕθ, θϕLpϕθ

(<tf.Tensor: id=988, shape=(3, 3), dtype=float32, numpy=
 array([[10., 11., 12.],
        [11., 13., 15.],
        [12., 15., 18.]], dtype=float32)>,
 <tf.Tensor: id=990, shape=(3, 3), dtype=float32, numpy=
 array([[10.,  9.,  8.],
        [ 9., 13., 17.],
        [ 8., 17., 26.]], dtype=float32)>)

In [146]:
loss = tf.linalg.trace(θϕLϕθ) / tf.linalg.trace(θϕLpϕθ)
loss

<tf.Tensor: id=997, shape=(), dtype=float32, numpy=0.8367347>